In [1]:
#!/usr/bin/env python3
"""
WebSocket Simulator para Kroner Hub
Simula el envío de datos BLE al servidor ESP32 via WebSocket
Basado en los logs de LOG_Consola.md
"""

import asyncio
import websockets
import json
import base64
import random
import time
from datetime import datetime

# Configuración del servidor
WEBSOCKET_URI = "ws://192.168.4.1:81/"
WIFI_SSID = "Kroner"

class KronerSimulator:
    """Simulador de mensajes para el Kroner Hub Display"""
    
    def __init__(self):
        self.ws = None
        self.running = False
        self.message_count = 0
        
    def format_display_message(self, msg_type: int, points: str, text: str) -> str:
        """
        Formatea un mensaje según el protocolo del display
        Formato: XXYYT F PPTEXT...
        
        Args:
            msg_type: 1=crono+puntos, 2=texto, 3=limpiar, 4=texto con limpieza
            points: Puntos (2 dígitos, e.g., "05", "91", "--")
            text: Texto a mostrar (e.g., "01:23" para crono)
        
        Returns:
            Mensaje formateado como string
        """
        # XXYY son fijos (según los logs parecen ser 0000)
        prefix = "0000"
        
        # T = tipo
        type_char = str(msg_type)
        
        # Separador
        separator = " "
        
        # PP = puntos (2 caracteres)
        points_str = points[:2].ljust(2, '-')
        
        # Construir mensaje
        message = f"{prefix}{type_char}{separator}{points_str}{text}"
        return message
    
    def create_websocket_payload(self, display_message: str) -> dict:
        """
        Crea el payload JSON para enviar por WebSocket
        
        Args:
            display_message: Mensaje formateado
            
        Returns:
            Diccionario con formato {"len": int, "time": int, "data": base64_str}
        """
        # Convertir mensaje a bytes
        message_bytes = display_message.encode('utf-8')
        
        # Codificar en base64
        base64_data = base64.b64encode(message_bytes).decode('utf-8')
        
        # Crear payload
        payload = {
            "len": len(message_bytes),
            "time": int(time.time() * 1000),  # timestamp en ms
            "data": base64_data
        }
        
        return payload
    
    async def send_message(self, msg_type: int, points: str, text: str):
        """Envía un mensaje al servidor WebSocket"""
        if not self.ws:
            print("❌ WebSocket no conectado")
            return
        
        # Formatear mensaje
        display_msg = self.format_display_message(msg_type, points, text)
        
        # Crear payload
        payload = self.create_websocket_payload(display_msg)
        
        # Enviar
        await self.ws.send(json.dumps(payload))
        
        self.message_count += 1
        timestamp = datetime.now().strftime("%H:%M:%S")
        print(f"[{timestamp}] #{self.message_count}: {display_msg}")
    
    async def simulate_chronometer(self):
        """Simula un cronómetro con puntos acumulados"""
        print("\n🏃 Simulación: Cronómetro con puntos")
        
        seconds = 0
        points = 0
        
        for _ in range(60):  # 60 segundos
            if not self.running:
                break
            
            # Formato de tiempo MM:SS
            minutes = seconds // 60
            secs = seconds % 60
            time_str = f"{minutes:02d}:{secs:02d}"
            
            # Incrementar puntos aleatoriamente
            if random.random() < 0.3:  # 30% probabilidad
                points += random.randint(1, 5)
            
            # Enviar mensaje tipo 1 (cronómetro con puntos)
            await self.send_message(1, f"{points:02d}", time_str)
            
            seconds += 1
            await asyncio.sleep(0.25)  # 4 mensajes por segundo (como en los logs)
    
    async def simulate_text_messages(self):
        """Simula mensajes de texto diversos"""
        print("\n📝 Simulación: Mensajes de texto")
        
        messages = [
            "READY",
            "START",
            "GO!",
            "FINISH",
            "WINNER",
            "GAME OVER",
            "NEXT ROUND",
            "PAUSE",
            "RESUME",
            "STOP"
        ]
        
        for msg in messages:
            if not self.running:
                break
            
            # Tipo 2: texto normal
            await self.send_message(2, "--", msg)
            await asyncio.sleep(1)
    
    async def simulate_countdown(self):
        """Simula cuenta regresiva"""
        print("\n⏱️ Simulación: Cuenta regresiva")
        
        for count in range(10, -1, -1):
            if not self.running:
                break
            
            time_str = f"00:{count:02d}"
            await self.send_message(1, "--", time_str)
            await asyncio.sleep(1)
        
        # Mensaje final
        await self.send_message(4, "--", "TIME UP!")
    
    async def simulate_score_update(self):
        """Simula actualizaciones de puntuación"""
        print("\n🎯 Simulación: Actualización de puntuación")
        
        for i in range(20):
            if not self.running:
                break
            
            points = random.randint(0, 99)
            time_display = f"{i//60:02d}:{i%60:02d}"
            
            await self.send_message(1, f"{points:02d}", time_display)
            await asyncio.sleep(0.5)
    
    async def simulate_mixed_scenario(self):
        """Simula un escenario mixto con diferentes tipos de mensajes"""
        print("\n🎲 Simulación: Escenario mixto")
        
        # Limpiar pantalla
        await self.send_message(3, "--", "")
        await asyncio.sleep(0.5)
        
        # Mensaje de inicio
        await self.send_message(4, "--", "READY")
        await asyncio.sleep(2)
        
        # Cronómetro corto
        for i in range(15):
            if not self.running:
                break
            time_str = f"00:{i:02d}"
            points = i * 2
            await self.send_message(1, f"{points:02d}", time_str)
            await asyncio.sleep(0.3)
        
        # Mensaje final
        await self.send_message(4, f"{i*2:02d}", "COMPLETE!")
        await asyncio.sleep(2)
    
    async def connect_and_run(self, simulation_mode="mixed"):
        """
        Conecta al WebSocket y ejecuta simulaciones
        
        Args:
            simulation_mode: "chrono", "text", "countdown", "score", "mixed", o "all"
        """
        print(f"🔌 Conectando a {WEBSOCKET_URI}...")
        print(f"📡 Asegúrate de estar conectado a la red WiFi: {WIFI_SSID}\n")
        
        try:
            async with websockets.connect(WEBSOCKET_URI) as websocket:
                self.ws = websocket
                self.running = True
                print("✅ Conectado exitosamente!\n")
                
                # Ejecutar simulación seleccionada
                if simulation_mode == "chrono":
                    await self.simulate_chronometer()
                elif simulation_mode == "text":
                    await self.simulate_text_messages()
                elif simulation_mode == "countdown":
                    await self.simulate_countdown()
                elif simulation_mode == "score":
                    await self.simulate_score_update()
                elif simulation_mode == "mixed":
                    await self.simulate_mixed_scenario()
                elif simulation_mode == "all":
                    # Ejecutar todas las simulaciones
                    await self.simulate_text_messages()
                    await asyncio.sleep(2)
                    await self.simulate_countdown()
                    await asyncio.sleep(2)
                    await self.simulate_score_update()
                    await asyncio.sleep(2)
                    await self.simulate_chronometer()
                
                print(f"\n✅ Simulación completada. Total mensajes: {self.message_count}")
                
        except ConnectionRefusedError:
            print(f"❌ No se pudo conectar a {WEBSOCKET_URI}")
            print(f"   Verifica que:")
            print(f"   1. Estás conectado a la red WiFi '{WIFI_SSID}'")
            print(f"   2. El servidor ESP32 está encendido")
            print(f"   3. El servidor WebSocket está corriendo en el puerto 81")
        except Exception as e:
            print(f"❌ Error: {e}")
        finally:
            self.running = False
            self.ws = None


# Para ejecutar desde el notebook
async def run_simulation(mode="mixed"):
    """
    Ejecuta la simulación con el modo especificado
    
    Modos disponibles:
    - "chrono": Cronómetro con puntos
    - "text": Mensajes de texto
    - "countdown": Cuenta regresiva
    - "score": Actualización de puntuación
    - "mixed": Escenario mixto (por defecto)
    - "all": Todas las simulaciones
    """
    simulator = KronerSimulator()
    await simulator.connect_and_run(simulation_mode=mode)

# Información de uso
print("=" * 70)
print("🎮 SIMULADOR WEBSOCKET KRONER HUB")
print("=" * 70)
print("\n📋 Instrucciones:")
print("1. Conecta tu computadora a la red WiFi 'Kroner'")
print("2. Abre el navegador en http://192.168.4.1 para ver el display")
print("3. Ejecuta una de las siguientes celdas para simular mensajes:")
print("\n💡 Modos de simulación disponibles:")
print("   • chrono    - Cronómetro con puntos acumulados")
print("   • text      - Mensajes de texto diversos")
print("   • countdown - Cuenta regresiva")
print("   • score     - Actualización de puntuación")
print("   • mixed     - Escenario mixto (recomendado)")
print("   • all       - Todas las simulaciones")
print("=" * 70)

🎮 SIMULADOR WEBSOCKET KRONER HUB

📋 Instrucciones:
1. Conecta tu computadora a la red WiFi 'Kroner'
2. Abre el navegador en http://192.168.4.1 para ver el display
3. Ejecuta una de las siguientes celdas para simular mensajes:

💡 Modos de simulación disponibles:
   • chrono    - Cronómetro con puntos acumulados
   • text      - Mensajes de texto diversos
   • countdown - Cuenta regresiva
   • score     - Actualización de puntuación
   • mixed     - Escenario mixto (recomendado)
   • all       - Todas las simulaciones


## 🎯 Simulación: Escenario Mixto (Recomendado)
Ejecuta un escenario completo con diferentes tipos de mensajes

In [6]:
# Ejecutar simulación mixta (escenario completo)
await run_simulation(mode="mixed")

🔌 Conectando a ws://192.168.4.1:81/...
📡 Asegúrate de estar conectado a la red WiFi: Kroner

✅ Conectado exitosamente!


🎲 Simulación: Escenario mixto
[13:22:49] #1: 00003 --
[13:22:50] #2: 00004 --READY
[13:22:52] #3: 00001 0000:00
[13:22:52] #4: 00001 0200:01
[13:22:53] #5: 00001 0400:02
[13:22:53] #6: 00001 0600:03
[13:22:53] #7: 00001 0800:04
[13:22:53] #8: 00001 1000:05
[13:22:54] #9: 00001 1200:06
[13:22:54] #10: 00001 1400:07
[13:22:54] #11: 00001 1600:08
[13:22:55] #12: 00001 1800:09
[13:22:55] #13: 00001 2000:10
[13:22:55] #14: 00001 2200:11
[13:22:56] #15: 00001 2400:12
[13:22:56] #16: 00001 2600:13
[13:22:56] #17: 00001 2800:14
[13:22:56] #18: 00004 28COMPLETE!


CancelledError: 

## ⏱️ Simulación: Cronómetro con Puntos
Simula un cronómetro que incrementa con puntuación variable

In [7]:
# Ejecutar simulación de cronómetro
await run_simulation(mode="chrono")

🔌 Conectando a ws://192.168.4.1:81/...
📡 Asegúrate de estar conectado a la red WiFi: Kroner

✅ Conectado exitosamente!


🏃 Simulación: Cronómetro con puntos
[13:23:03] #1: 00001 0000:00
[13:23:04] #2: 00001 0000:01
[13:23:04] #3: 00001 0000:02
[13:23:04] #4: 00001 0000:03
[13:23:04] #5: 00001 0300:04
[13:23:05] #6: 00001 0300:05
[13:23:05] #7: 00001 0300:06
[13:23:05] #8: 00001 0300:07
[13:23:05] #9: 00001 0800:08
[13:23:06] #10: 00001 1000:09
[13:23:06] #11: 00001 1000:10
[13:23:06] #12: 00001 1300:11
[13:23:06] #13: 00001 1300:12
[13:23:07] #14: 00001 1300:13
[13:23:07] #15: 00001 1300:14
[13:23:07] #16: 00001 1300:15
[13:23:07] #17: 00001 1300:16
[13:23:08] #18: 00001 1300:17
[13:23:08] #19: 00001 1300:18
[13:23:08] #20: 00001 1300:19
[13:23:08] #21: 00001 1300:20
[13:23:09] #22: 00001 1700:21
[13:23:09] #23: 00001 1800:22
[13:23:09] #24: 00001 1800:23
[13:23:09] #25: 00001 1800:24
[13:23:10] #26: 00001 1800:25
[13:23:10] #27: 00001 1800:26
[13:23:10] #28: 00001 1800:27
[13:23:10] #

CancelledError: 

## 📝 Simulación: Mensajes de Texto
Muestra diferentes mensajes de texto en el display

In [8]:
# Ejecutar simulación de mensajes de texto
await run_simulation(mode="text")

🔌 Conectando a ws://192.168.4.1:81/...
📡 Asegúrate de estar conectado a la red WiFi: Kroner

✅ Conectado exitosamente!


📝 Simulación: Mensajes de texto
[13:23:25] #1: 00002 --READY
[13:23:26] #2: 00002 --START
[13:23:27] #3: 00002 --GO!
[13:23:28] #4: 00002 --FINISH


CancelledError: 

## ⏳ Simulación: Cuenta Regresiva
Cuenta regresiva de 10 a 0

In [9]:
# Ejecutar cuenta regresiva
await run_simulation(mode="countdown")

🔌 Conectando a ws://192.168.4.1:81/...
📡 Asegúrate de estar conectado a la red WiFi: Kroner

✅ Conectado exitosamente!


⏱️ Simulación: Cuenta regresiva
[13:23:32] #1: 00001 --00:10
[13:23:33] #2: 00001 --00:09
[13:23:34] #3: 00001 --00:08
[13:23:35] #4: 00001 --00:07


CancelledError: 

## 🎯 Simulación: Actualización de Puntuación
Actualiza la puntuación en tiempo real

In [10]:
# Ejecutar simulación de puntuación
await run_simulation(mode="score")

🔌 Conectando a ws://192.168.4.1:81/...
📡 Asegúrate de estar conectado a la red WiFi: Kroner

✅ Conectado exitosamente!


🎯 Simulación: Actualización de puntuación
[13:23:39] #1: 00001 4900:00
[13:23:40] #2: 00001 6100:01
[13:23:40] #3: 00001 1800:02
[13:23:41] #4: 00001 3400:03


CancelledError: 

## 🎪 Simulación: Todas las Simulaciones
Ejecuta todas las simulaciones secuencialmente

In [ ]:
# Ejecutar todas las simulaciones
await run_simulation(mode="all")